In [ ]:
import os
import xarray as xr
import geopandas as gpd
import pandas as pd
from datetime import datetime
!pip install rasterstats
from rasterstats import zonal_stats
import numpy as np
from shapely.geometry import Point, box
from rasterio.features import geometry_mask
import time
import xarray as xr
from shapely.geometry import mapping
import rioxarray
from scipy.stats import genextreme
from concurrent.futures import ProcessPoolExecutor
import gcsfs  # For interacting with Google Cloud Storage
import tempfile
import shutil
from tempfile import TemporaryDirectory
from google.cloud import storage
import sys, pathlib
from pathlib import Path
sys.path.insert(0, "src")
from MSWEP_Preprocess import *

# Input Files

In [ ]:
# Define your bucket and paths
bucket = 'gs://leap-persistent/adamnayak/flood-insurance'
county_shapefile = '../../Local_Data/Geospatial/tl_2019_us_county.shp'
precipitation_nc_path = 'MSWEP/Daily_Annual/MSWEP_1979.nc'
weights_output_path = 'MSWEP/MSWEP_county_pixel_weights.npy'

# Calculate County Weights

In [ ]:
# Call the function
weights = calculate_pixel_weights_mSWEP(
    county_shapefile=county_shapefile,
    precipitation_nc_path=precipitation_nc_path,
    output_path=weights_output_path,
    gcs_bucket=bucket
)

# Aggregate Daily Precipitation

In [ ]:
# Define your paths
precipitation_dir = 'MSWEP/Daily_Annual/'
output_dir = 'MSWEP/County_Daily_Precip_By_Year'

In [ ]:
# Call the function
process_mSWEP_precipitation(
    county_shapefile=county_shapefile,
    precipitation_dir=precipitation_dir,
    weights=weights,
    output_dir=output_dir,
    gcs_bucket=bucket
)

# Rechunk to County

In [ ]:
# Define GCS bucket and paths
input_path = 'MSWEP/County_Daily_Precip_By_Year'
output_path = 'MSWEP/County_Daily_Precip_By_County'

# Define the range of years to process
precip_years = range(1979, 2021)

In [ ]:
rechunk_precipitation_data_by_county_gcs(precip_years, 
                                         input_path, 
                                         output_path, 
                                         bucket)

# Calculate County Precip Stats of Interest

In [ ]:
bucket_name = 'leap-persistent'
source_folder = 'adamnayak/flood-insurance/MSWEP/County_Daily_Precip_By_County'
local_folder = 'MSWEP_County_Precip_Local'

In [ ]:
# Download data from GCS
download_from_gcs(bucket_name, source_folder, local_folder)

In [ ]:
# Process all precipitation data
process_all_precipitation_data_mswep(local_folder)

# Merge with ERA5 Processed Data

In [ ]:
# Define folders for analysis
claims_folder = 'ERA5_Merged_Claims'
precip_folder = 'MSWEP_Daily_Precip_Processed_County'
output_folder = 'MSWEP_ERA5_Merged_Claims'

In [ ]:
# Merge claims with precipitation data
merge_claims_with_precip(claims_folder=claims_folder, 
                         precip_folder=precip_folder, 
                         output_folder=output_folder)

# Concatenate Claims with Precip

In [ ]:
processed_path="MSWEP_ERA5_Merged_Claims"
output_file="MSWEP_ERA5_Processed_Claims.csv"

In [ ]:
concatenate_processed_claims(output_file=output_file, 
                                 processed_path=processed_path)

# Clean Up

In [ ]:
# Define bucket and folder paths
bucket_name = "leap-persistent"
target_folder = "adamnayak/flood-insurance/MSWEP"  # Updated path

In [ ]:
local_paths = ['MSWEP_Ann_Max_Precip_County', 'MSWEP_Daily_Precip_Processed_County', 'MSWEP_ERA5_Merged_Claims', 'MSWEP_ERA5_Processed_Claims.csv']
remove_flags = [True, True, False, False]  # Specify which items to remove after upload

push_to_cloud_and_conditional_remove(local_paths, remove_flags, bucket_name, target_folder)

In [ ]:
!rm -r MSWEP_County_Precip_Local
!rm -r ERA5_Merged_Claims
# Note that we want to keep MSWEP_ERA5_Merged_Claims folder until all datasets are merged to the claims